## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Visualizar dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Dividir dataset en rasgos y clases

In [3]:
x = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

## Preprocesamiento de datos

In [4]:
# --> Valores categoricos para varias columnas para paises

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# --> Transformar varias columnas
ct = ColumnTransformer(
    # Columnas para transformar
    [('one_hot_encoder1', OneHotEncoder(), [1])],
    # Lo que pasara con las demas columnas
    remainder="passthrough"
)

x = ct.fit_transform(x)

In [5]:
# --> Para columna de genero

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

x[:, 4] = label_encoder.fit_transform(x[:, 4])

In [6]:
# --> Solucionar problema de multicolinealidad(eliminar una de las variables dummy)
x = x[:, 1:]

## Dividir en entrenamiento y prueba

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
x_train

array([[0.0, 1.0, 667, ..., 1, 0, 163830.64],
       [1.0, 0.0, 427, ..., 1, 1, 57098.0],
       [0.0, 0.0, 535, ..., 1, 0, 185630.76],
       ...,
       [0.0, 0.0, 738, ..., 1, 0, 181429.87],
       [0.0, 1.0, 590, ..., 1, 1, 148750.16],
       [1.0, 0.0, 623, ..., 1, 0, 118855.26]], dtype=object)

## Escalado de variables

In [8]:
from sklearn.preprocessing import StandardScaler

# --> Objeto para estandarizar
sc_x = StandardScaler()

# --> Aplicar estandarizacion
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)


## Construir la red neuronal artificial

In [9]:
# --> Librerias

import keras
from keras.models import Sequential     # Red neuronal por capas
from keras.layers import Dense          # Capas densas

In [21]:
# --> Inicializar red neuronal

classifier = Sequential()

In [22]:
# --> Capa de entrada y primera capa oculta
# Para los nodos de salida = (nodos de entrada + 1) / 2

classifier.add(Dense(
    units=6,                        # Numero de nodos de salida 
    kernel_initializer='uniform',   # Inicializacion de los pesos
    activation='relu',              # Funcion de activacion
    input_dim=11                    # Numero de nodos de entrada
    ))

In [23]:
# --> Segunda capa oculta

classifier.add(Dense(
    units=6,                        # Numero de nodos de salida
    kernel_initializer='uniform',   # Inicializacion de los pesos
    activation='relu',              # Funcion de activacion
))


In [25]:
# --> Terdera capa oculta

classifier.add(Dense(
    units=6,                        # Numero de nodos de salida
    kernel_initializer='uniform',   # Inicializacion de los pesos
    activation='relu',              # Funcion de activacion
))

In [26]:
# --> Capa de salida
# Dado que regresara 1 o 0 entonces las capas de salida sera solo 1, si fuera a clasificar 3 clases entonces las capas son 3 y la función de activación cambiaría

classifier.add(Dense(
    units=1,                        # Numero de nodos de salida
    kernel_initializer='uniform',   # Inicializacion de los pesos
    activation='sigmoid',           # Funcion de activacion
))

In [27]:
# --> Compilar red neuronal

classifier.compile(
    optimizer='adam',               # Algoritmo de optimizacion
    loss='binary_crossentropy',     # Funcion de perdida
    metrics=['accuracy']            # Metrica de evaluacion
)

## Entrenar red neuronal

In [28]:
# --> Ajustar red neuronal

classifier.fit(
    x_train,                        # Datos de entrenamiento
    y_train,                        # Salida de los datos de entrenamiento
    batch_size=10,                  # Tamaño de los lotes
    epochs=50                       # Numero de epocas
)

Epoch 1/50
800/800 [==============================] - 2s 2ms/step - loss: 0.6138 - accuracy: 0.7956
Epoch 2/50
800/800 [==============================] - 1s 2ms/step - loss: 0.5340 - accuracy: 0.7960
Epoch 3/50
800/800 [==============================] - 1s 2ms/step - loss: 0.5119 - accuracy: 0.7960
Epoch 4/50
800/800 [==============================] - 1s 2ms/step - loss: 0.5069 - accuracy: 0.7960
Epoch 5/50
800/800 [==============================] - 1s 2ms/step - loss: 0.5061 - accuracy: 0.7960
Epoch 6/50
800/800 [==============================] - 1s 2ms/step - loss: 0.5060 - accuracy: 0.7960
Epoch 7/50
800/800 [==============================] - 1s 2ms/step - loss: 0.5059 - accuracy: 0.7960
Epoch 8/50
800/800 [==============================] - 1s 2ms/step - loss: 0.5059 - accuracy: 0.7960
Epoch 9/50
800/800 [==============================] - 1s 2ms/step - loss: 0.5059 - accuracy: 0.7960
Epoch 10/50
800/800 [==============================] - 1s 2ms/step - loss: 0.5059 - accuracy: 0.7960

## Predicción de valores

In [29]:
y_predict = classifier.predict(x_test)  # Prediccion de los datos de prueba
y_predict = (y_predict > 0.5)           # Si es mayor a 0.5 entonces es 1, si no es 0

63/63 [==============================] - 0s 2ms/step


## Matriz de confusión

In [30]:
# --> Matriz de confusion

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_predict)
cm

array([[1595,    0],
       [ 405,    0]], dtype=int64)

In [31]:
print('Accuracy: ', round(100*(cm[0][0] + cm[1][1]) / len(y_test), 2), '%')

Accuracy:  79.75 %
